In [ ]:
import pandas as pd
import numpy as np
import re as re
from deep_translator import GoogleTranslator
import textblob as tb

Lecture de données JSON

In [ ]:
pdTweet = pd.read_json(path_or_buf="versailles_tweets_100.json")
pdTweet

<H3> Partie nettoyage du texte <H3/>

Retire les retours à la ligne:

In [ ]:

def no_retourchariot(chiffre):
    """Fonction qui retire les retours à la ligne du texte"""
    patern = re.compile("\n")
    result = patern.sub(r"",pdTweet["text"][chiffre])
    return result

Extraction des tweet sans emoticônes (Expressions régulières)


Source: https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b?permalink_comment_id=3315605#gistcomment-3315605

In [ ]:

def no_emoticone(chiffre):
        """fonction qui retire tout les émoticones du tweet"""
        patern  = re.compile("["
                        u"\U0001F600-\U0001F64F"  # emoticons
                        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                        u"\U0001F680-\U0001F6FF"  # transport & map symbols
                        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                        u"\U00002500-\U00002BEF"  # chinese char
                        u"\U00002702-\U000027B0"
                        u"\U00002702-\U000027B0"
                        u"\U000024C2-\U0001F251"
                        u"\U0001f926-\U0001f937"
                        u"\U00010000-\U0010ffff"
                        u"\u2640-\u2642"
                        u"\u2600-\u2B55"
                        u"\u200d"
                        u"\u23cf"
                        u"\u23e9"
                        u"\u231a"
                        u"\ufe0f"  # dingbats
                        u"\u3030"
                                "]+", flags = re.UNICODE)
        
        result = patern.sub(r"",chiffre)
        return result

Retire mention du texte d'un tweet

In [ ]:
def no_mention(chiffre):
    """Fonction qui retire les mentions du tweet"""
    patern = re.compile("@\S*")
    result = patern.sub(r"",chiffre)
    return result

Retire les hastags d'un tweet

In [ ]:
def no_hastags(chiffre):
    """fonction qui retire les hastags du tweet"""
    patern = re.compile("#\S*")
    result = patern.sub(r"",chiffre)
    return result

Retire l'intégration d'un URL dans un tweet

In [ ]:

def no_URL(chiffre):
    """fonction qui retire l'URL d'un tweet"""
    patern = re.compile("https://\S*")
    result = patern.sub(r"", chiffre)
    return result


Fonction qui nettoie le texte avec toutes les fonctions créé auparavant.

In [ ]:
def menage(NumeroTweet):
    "fonction qui applique toutes les autres fonction pour avoir un texte propre"
    result = no_retourchariot(NumeroTweet)
    result = no_emoticone(result)
    result = no_mention(result)
    result = no_hastags(result)
    result = no_URL(result)
    return result

In [ ]:
pdTweetFiltre = pd.DataFrame()

Insérer le texte nettoyer des tweet dans le data Frame TweetFiltrer

In [ ]:
#insertion du texte nettoyer dans le Data Frame TweetFiltrer
list_texte = []
for i in range(0,20):
    texte = menage(i)
    list_texte.append(texte)
pdTweetFiltre["Texte"] = list_texte
pdTweetFiltre

<H3>Partie gestion Mentions URl et Hashtags<H3/>

In [ ]:
# Crée un dataFrame avec les données inscrites dans entites, en columns on a Url, Mentions et Hashtags
pdEntities = pdTweet["entities"].apply(pd.Series)
# On supprime la column 0 qui ne contient rien.
del pdEntities[0]
pdEntities

Cette cellule trie les informations stockées dans la colonnes "Entities" dans le dataframe initial et le stocke dans un autre dataframe.

In [ ]:
for columns_name in pdEntities:
        lenSeries = len(pdEntities[columns_name])
        lElement = []
        if columns_name == "hashtags":
                key = "tag"
        elif columns_name == "mentions":
                key = "username"
        elif columns_name == "urls":
                key = "url"
        else:
                key = None
        for i in range(0,lenSeries):
                lElementUnder = []
                if type(pdEntities[columns_name][i]) == list:
                        for j in range(0, int(len(pdEntities[columns_name][i]))):
                                lElementUnder.append(pdEntities[columns_name][i][j].get(key))
                else:
                        lElementUnder.append(None)
                lElement.append(lElementUnder)
        lElement = pd.Series(lElement)
        pdTweetFiltre[columns_name] = lElement
pdTweetFiltre

On rajoute les id des auteurs dans le dataframe

In [ ]:
pdTweetFiltre["author"] = pdTweet["author_id"]
pdTweetFiltre

On fait les mêmes manipulations pour les topics.

In [ ]:
sTopics = []
for ligne in pdTweet["context_annotations"]:
    sTopicsTweet = []
    if type(ligne) == list:
        lenSeries = len(ligne)
        for i in range(0, lenSeries):
            topics = ligne[i].get("domain").get("name")
            if topics not in sTopicsTweet:
                sTopicsTweet.append(topics)
    else:
        sTopicsTweet.append(None)
    sTopics.append(sTopicsTweet)
pdTweetFiltre["topics"] = sTopics
pdTweetFiltre

Utilisation de la libraire textBlob pour donner le sentiment (négatif ou positif) du tweet à partir de son texte.

On utilise l'api de Googletranslate via la libraire deep-translate pour traduire le texte auparavant.

In [ ]:
def analysis_sentiment(source:pd.DataFrame):
    translator = GoogleTranslator(target="en")
    liste_sentiment = []
    for i in range(0, len(source["Texte"])):
        texte = source["Texte"][i]
        texte_Angl = translator.translate(texte)
        liste_sentiment.append(tb.TextBlob(texte_Angl).sentiment[0])
    return liste_sentiment

pdTweetFiltre["sentiment"] = analysis_sentiment(pdTweetFiltre)

Affichage des tweets d'un utilisateur :

In [ ]:
def tweet_author (author):
    """fonction qui affiche tout les tweets d'un utilisateur donné"""
    serie = pd.Series([author for i in range(len(pdTweetFiltre["author"]))])
    return pdTweetFiltre[pdTweetFiltre["author"].isin(serie)]

tweet_author(1339914264522461184)

Affichage de l'ensemble des tweets mentionnant un utilisateur spécifique :

In [ ]:
def tweet_mention (mention):
    "fonction qui affiche le texte de ou des tweets nettoyer ayant la mention de la personne dedans"
    for i in range(0, len(pdTweetFiltre["mentions"])):
        liste_mention = pdTweetFiltre["mentions"][i]
        for j in range(0, len(liste_mention)):
            mention_tab = liste_mention[j]
            if mention == mention_tab :
                print(pdTweetFiltre["Texte"][i] + "\n")

tweet_mention("leonna_julie")

Affichage des Utilisateurs utilisant un Hashtag spécifique :

In [ ]:
def tweet_hashtags (hashtag):
    """fonction qui va afficher le ou les utilisateurs ayants utilisés le hastags dans leur tweet"""
    for i in range(0, len(pdTweetFiltre["hashtags"])):
        liste_hashtags = pdTweetFiltre["hashtags"][i]
        for j in range(0, len(liste_hashtags)):
            mention_tab = liste_hashtags[j]
            if hashtag == mention_tab :
                print(pdTweetFiltre["author"][i])

tweet_hashtags("versailles")

Affichage des utilisateurs mentionnés dans les tweets d'un autre utilisateur :

In [ ]:
def author_mention(author):
    """fonction qui affiche les mentions q'un utilisateur a fait dans ses tweets"""
    liste_mention = 0
    mention =[]
    for i  in range(0, len(tweet_author(author))):
        liste_mention = (tweet_author(author)["mentions"][i])
        if liste_mention != [None]:
            for j in range(0,len(liste_mention)):
                mention.append(liste_mention[j])
    return set(mention)

    


author_mention(1339914264522461184)

<H4>Top K hashtags:</H4>

On va lire dans le dataFrame qu'on a créé les différents hashtags présent.
Pour chaque hashtags, si il est dans la liste des clés déja renseigné, on ajoute + 1 a la valeur de la clé. Sinon, on le rajoute dans la liste des clés et on l'initialise à 1.

On va ensuite obtenir les valeurs les plus grandes du dictionnaire avec la fonction max. Si la valeur est représenté plusieurs fois (2ième exaquo) on montrera toutes les valeurs qui correpondent au maximum.

Les données sont organisées de la même manière pour les mentions et les url. On peut donc créer une fonction qui prend 3 arguments: k qui correspond le nombre de valeurs à afficher, source qui est le dataframe duquel on va afficher le classement et entities qui est une variable pour faire le choix entre mentions, hashtags et topics.

In [ ]:
def extraire_dict(entities:str, source:pd.DataFrame):
    """
    Cette fonction permet d'extraire les donnnées du dataframe sous forme de dictionnaire.
    On peut utiliser dans "entities" les arguments: Hashtags, mentions, author et topics.
    Author est vu comme une exception car il n'est pas stocké de la même manière que les autres.
    """
    dict_Entities = {}
    series_Entities = source[entities]
    if entities == "author":
        dataFrame_Top = pd.DataFrame(series_Entities.value_counts())
        dict_Entities = dataFrame_Top.to_dict()["author"]
    else:
        for tweet in series_Entities:
            for i in range(0, len(tweet)):
                value = dict_Entities.get(tweet[i])
                # Value représente le nombre de fois que le hashtags a déjà été lu dans le DataFrame.
                # Si c'est la première fois qu'on le voit, value retourne None et donc on initialise à 1
                if value != None:
                    dict_Entities[tweet[i]] = value + 1
                else:
                    dict_Entities[tweet[i]] = 1
        # On supprime la colonne qui montre les tweets sans hashtags
        del dict_Entities[None]
    return dict_Entities

In [ ]:
def top(k:int, source:pd.DataFrame, entities:str):
    """
    On prend dans source (qui doit être un data frame) le podium de l'entités demandé,
    L'entité peut être les hashtags, les utilisateurs, les topics ou encore les mentions.
    On réutilise la fonction créer avant pour obtenir le dictionnaire, puis on applique le trie
    pour afficher le top.
    """
    dict_Entities = extraire_dict(entities, source)
    dict_Top = {}
    # On veut le top k 
    for top in range(1, k + 1):
        if dict_Entities.values().__len__() != 0:
            dict_Top[top] = []
            maxRepresenter = max(dict_Entities.values())
            for key, values in dict_Entities.items():
                if values == maxRepresenter:
                    dict_Top[top].append(key)
            for candidat in dict_Top[top]:
                del dict_Entities[candidat]
    dataFrame_Top = pd.DataFrame()
    for top, hashtags in dict_Top.items():
        # Pour éviter les problèmes de tailles de colonnes, on va créer un df éphémère à chaque fois
        dfEphemere = pd.DataFrame()
        dfEphemere[top] = pd.Series(hashtags)
        # Puis on va le joindre à l'ancien df.
        dataFrame_Top = pd.concat([dataFrame_Top, dfEphemere], join="outer", axis=1)
    return dataFrame_Top

In [ ]:
top(3, pdTweetFiltre, "hashtags")

Pour obtenir le nombre de publication d'un utilisateur, le nombre de publications qui comporte un hashtags en particulier ou encore le nombre de publication par topics, on prend la fonction qui permet de nous donner un dictionnaire et on cherche dans ce dictionnaire la clé qu'il nous intéresse.

In [ ]:
def nbr_publi(name, source:pd.DataFrame, domaine:str):
    """
    Fonction qui retourne le nombre de publication dans un domaine. Le domain peut-être soit author, soit hashtags, soit mentions.
    La variable name définit soit l'id de l'utilisateur, soit le hashtags soit le nom du topic dans le cas d'une recherche 
    par topics.
    """
    return extraire_dict(domaine, source).get(name)

In [ ]:
nbr_publi("CIV", pdTweetFiltre, domaine="hashtags")